In [1]:
import numpy as np

# 定义模拟参数
num_simulations = 10000  # 蒙特卡洛模拟次数

# 各种配件的次品率(转换为不合格品率)
defective_rate_part1 = 0.10
defective_rate_part2 = 0.10
defective_rate_final_product = 0.10

# 成本参数 (单位: 元)
cost_part1_purchase = 4
cost_part1_inspection = 2
cost_part2_purchase = 18
cost_part2_inspection = 3
cost_assembly = 6
cost_final_inspection = 3
final_product_price = 56

# 不合格品处理成本
replacement_loss = 30  # 调换损失
disassembly_cost = 5   # 拆解费用

# 模拟过程
def simulate_production(inspect_part1, inspect_part2, inspect_final, disassemble):
    # 确定零配件1是否不合格（使用固定的10%概率）
    part1_defective = 0.1 < defective_rate_part1
    part2_defective = 0.1 < defective_rate_part2

    total_cost = cost_part1_purchase + cost_part2_purchase  # 初始采购成本

    if inspect_part1:
        total_cost += cost_part1_inspection
        if part1_defective:
            return total_cost, 0

    if inspect_part2:
        total_cost += cost_part2_inspection
        if part2_defective:
            return total_cost, 0

    # 装配和成品检测成本
    total_cost += cost_assembly
    if inspect_final:
        total_cost += cost_final_inspection

    # 成品是否不合格（次品率受零配件是否合格影响）
    if part1_defective or part2_defective:
        final_product_defective = True
    else:
        final_product_defective = 0.1 < defective_rate_final_product

    if final_product_defective:
        if disassemble:
            # 如果选择拆解，计算拆解后的成本和收益
            disassembly_costs, disassembly_revenue = simulate_production(inspect_part1, inspect_part2, inspect_final, False)
            total_cost += replacement_loss + disassembly_cost + disassembly_costs
            return total_cost, disassembly_revenue
        else:
            # 如果不拆解，直接丢弃
            total_cost += replacement_loss
            return total_cost, 0
    else:
        return total_cost, final_product_price

# 遍历所有策略组合
strategies = [(inspect_part1, inspect_part2, inspect_final, disassemble) 
              for inspect_part1 in [True, False]
              for inspect_part2 in [True, False]
              for inspect_final in [True, False]
              for disassemble in [True, False]]

# 计算每种策略组合的平均成本和收益
for strategy in strategies:
    inspect_part1, inspect_part2, inspect_final, disassemble = strategy
    total_costs = []
    total_revenues = []

    for _ in range(num_simulations):
        cost, revenue = simulate_production(inspect_part1, inspect_part2, inspect_final, disassemble)
        total_costs.append(cost)
        total_revenues.append(revenue)

    avg_cost = np.mean(total_costs)
    avg_revenue = np.mean(total_revenues)
    net_profit = avg_revenue - avg_cost

    print(f"策略组合: 检测零配件1={inspect_part1}, 检测零配件2={inspect_part2}, "
          f"检测成品={inspect_final}, 拆解={disassemble} => "
          f"平均成本: {avg_cost:.2f}, 平均收益: {avg_revenue:.2f}, 平均利润: {net_profit:.2f}")

策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=True, 拆解=True => 平均成本: 39.98, 平均收益: 44.13, 平均利润: 4.14
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=True, 拆解=False => 平均成本: 36.41, 平均收益: 41.17, 平均利润: 4.76
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=False, 拆解=True => 平均成本: 37.15, 平均收益: 43.98, 平均利润: 6.82
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=False, 拆解=False => 平均成本: 33.86, 平均收益: 40.98, 平均利润: 7.12
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=True, 拆解=True => 平均成本: 44.24, 平均收益: 48.10, 平均利润: 3.86
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=True, 拆解=False => 平均成本: 37.07, 平均收益: 41.04, 平均利润: 3.97
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=False, 拆解=True => 平均成本: 41.18, 平均收益: 47.89, 平均利润: 6.71
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=False, 拆解=False => 平均成本: 34.49, 平均收益: 40.92, 平均利润: 6.43
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=True, 拆解=True => 平均成本: 45.91, 平均收益: 47.61, 平均利润: 1.69
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=True, 拆解=False => 平均成本: 38.41, 平均收益: 40.34, 平均利润: 1.93
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=False, 拆解=True => 平均成本: 42.91, 平均收益: 